In [1]:
# Cell 1: Import libraries
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [20]:
# Cell 2: Configure Chrome with options to bypass detection
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)  # Prevents auto-close

# Add options to make selenium harder to detect
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# Start Selenium
driver = webdriver.Chrome(options=options)

# Execute CDP commands to bypass detection
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
    """
})

# Navigate to HackerRank
driver.get("https://www.hackerrank.com/dashboard")

In [21]:
# Cell 4: Navigate to Administration
try:
    # Navigate to Administration page using the CSS selector
    admin_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.profile-nav-item-link[href='/administration/contests']"))
    )
    admin_link.click()
    print("Successfully navigated to Administration page!")
except Exception as e:
    print(f"Error navigating to Administration page: {e}")

Error navigating to Administration page: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF70CF74C25+3179557]
	(No symbol) [0x00007FF70CBD88A0]
	(No symbol) [0x00007FF70CA691CA]
	(No symbol) [0x00007FF70CABFA67]
	(No symbol) [0x00007FF70CABFC9C]
	(No symbol) [0x00007FF70CB13627]
	(No symbol) [0x00007FF70CAE7C6F]
	(No symbol) [0x00007FF70CB102F3]
	(No symbol) [0x00007FF70CAE7A03]
	(No symbol) [0x00007FF70CAB06D0]
	(No symbol) [0x00007FF70CAB1983]
	GetHandleVerifier [0x00007FF70CFD67CD+3579853]
	GetHandleVerifier [0x00007FF70CFED1D2+3672530]
	GetHandleVerifier [0x00007FF70CFE2153+3627347]
	GetHandleVerifier [0x00007FF70CD4092A+868650]
	(No symbol) [0x00007FF70CBE2FFF]
	(No symbol) [0x00007FF70CBDF4A4]
	(No symbol) [0x00007FF70CBDF646]
	(No symbol) [0x00007FF70CBCEAA9]
	BaseThreadInitThunk [0x00007FFBF2B6E8D7+23]
	RtlUserThreadStart [0x00007FFBF3BFBF6C+44]


In [23]:
# Cell 5: Select specific contest using data-key
try:
    # Find the contest with the specific data-key
    contest_selector = f"td[data-key='308429.0']"
    
    # Wait for the element to be clickable
    contest_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, contest_selector))
    )
    
    # Click on the row to select the contest
    contest_element.click()
    print("Successfully selected the contest!")
    
except Exception as e:
    print(f"Error selecting contest: {e}")
    # Take a screenshot to debug
    driver.save_screenshot("contest_selection_error.png")
    print("Screenshot saved as 'contest_selection_error.png'")

Successfully selected the contest!


In [24]:
# Cell 6 (Alternative): Click on Statistics button using XPath with text
try:
    # Use XPath to find the element with the exact text "Statistics"
    statistics_xpath = "//a[contains(@class, 'cursor') and contains(@class, 'change-tab') and text()='Statistics']"
    
    # Wait for the element to be clickable
    statistics_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, statistics_xpath))
    )
    
    # Click the Statistics button
    statistics_button.click()
    print("Successfully clicked on the Statistics button!")
    
except Exception as e:
    print(f"Error clicking Statistics button using XPath: {e}")
    # Take a screenshot to debug
    driver.save_screenshot("statistics_xpath_error.png")
    print("Screenshot saved as 'statistics_xpath_error.png'")

Successfully clicked on the Statistics button!


In [25]:
# Cell 7: Navigate to contest submissions page
try:
    # Use the CSS selector for the "View all contest submissions" link
    submissions_selector = "a.mlT.btn.btn-green"
    
    # Wait for the element to be clickable
    submissions_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, submissions_selector))
    )
    
    # Click the submissions link
    submissions_link.click()
    print("Successfully navigated to the contest submissions page!")
    
except Exception as e:
    print(f"Error navigating to submissions page: {e}")
    # Take a screenshot to debug
    driver.save_screenshot("submissions_click_error.png")
    print("Screenshot saved as 'submissions_click_error.png'")

Successfully navigated to the contest submissions page!


In [26]:
import csv
import time
import os
import random
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

def scrape_hackerrank_with_pagination(driver, max_pages=190, batch_size=3, 
                                     output_dir="hackerrank_data", 
                                     wait_between_pages=(5, 10)):
    """
    Scrape HackerRank submissions with pagination, carefully managing resources
    
    Args:
        driver: Selenium WebDriver instance (already on the submissions page)
        max_pages: Maximum number of pages to scrape (default 190)
        batch_size: Number of submissions to open at once (default 3)
        output_dir: Directory to save output files
        wait_between_pages: Random wait range between pages (min, max) in seconds
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create a log file
    log_file = os.path.join(output_dir, f"scrape_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
    
    def log_message(message):
        """Write message to log file and print to console"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] {message}"
        print(log_entry)
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(log_entry + "\n")
    
    # Initialize count of total submissions scraped
    total_submissions = 0
    
    # Process each page
    for page_num in range(1, max_pages + 1):
        log_message(f"\n==== Processing Page {page_num}/{max_pages} ====")
        
        # Save a page progress file to enable resuming if needed
        with open(os.path.join(output_dir, "page_progress.txt"), "w") as f:
            f.write(f"last_page_started: {page_num}\n")
        
        # Find all view buttons on the current page
        try:
            # Wait for page to fully load with sufficient timeout
            time.sleep(3)  # Initial pause to let page elements settle
            
            view_buttons = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.view-results"))
            )
            log_message(f"Found {len(view_buttons)} 'View' buttons on page {page_num}")
            
            # Store the main window handle
            main_window = driver.current_window_handle
            
            # Process submissions in smaller batches to manage memory
            page_submissions = []
            for batch_start in range(0, len(view_buttons), batch_size):
                batch_end = min(batch_start + batch_size, len(view_buttons))
                batch_buttons = view_buttons[batch_start:batch_end]
                
                log_message(f"Processing batch {batch_start//batch_size + 1} of {(len(view_buttons) + batch_size - 1)//batch_size}")
                
                # Open each submission in this batch in a new tab
                for i, button in enumerate(batch_buttons):
                    try:
                        url = button.get_attribute("href")
                        submission_id = url.split("/code/")[1] if "/code/" in url else "unknown"
                        
                        # Open in new tab using JavaScript
                        driver.execute_script("arguments[0].setAttribute('target', '_blank');", button)
                        button.click()
                        time.sleep(1)  # Pause between opening tabs
                        log_message(f"Opened submission {batch_start + i + 1}/{len(view_buttons)} (ID: {submission_id})")
                    except Exception as e:
                        log_message(f"Error opening tab for submission {batch_start + i + 1}: {e}")
                
                # Wait for all tabs to finish loading
                time.sleep(5)  # Give time for tabs to start loading
                
                # Get all window handles (tabs)
                all_tabs = driver.window_handles
                submission_tabs = [tab for tab in all_tabs if tab != main_window]
                
                # Process each submission tab in this batch
                for tab in submission_tabs:
                    try:
                        # Switch to the tab
                        driver.switch_to.window(tab)
                        
                        # Wait for the code container to load with a generous timeout
                        WebDriverWait(driver, 20).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "div.CodeMirror-lines[role='presentation']"))
                        )
                        
                        # Extract submission details
                        current_url = driver.current_url
                        submission_id = current_url.split("/code/")[1] if "/code/" in current_url else "unknown"
                        
                        # Try multiple selectors for username
                        username = "unknown"
                        for selector in [".submitter-details .username", ".profile-username", ".submitter-name"]:
                            try:
                                username_element = driver.find_element(By.CSS_SELECTOR, selector)
                                if username_element.text:
                                    username = username_element.text
                                    break
                            except:
                                continue
                        
                        # Try multiple selectors for problem name
                        problem_name = "unknown"
                        for selector in [".problem-title", ".challenge-name", ".challenge-title"]:
                            try:
                                problem_element = driver.find_element(By.CSS_SELECTOR, selector)
                                if problem_element.text:
                                    problem_name = problem_element.text
                                    break
                            except:
                                continue
                        
                        # Try multiple selectors for language
                        language = "unknown"
                        for selector in [".language-name", ".select-language", ".language-selector"]:
                            try:
                                language_element = driver.find_element(By.CSS_SELECTOR, selector)
                                if language_element.text:
                                    language = language_element.text
                                    break
                            except:
                                continue
                        
                        # Extract the code
                        code_container = driver.find_element(By.CSS_SELECTOR, "div.CodeMirror-lines[role='presentation']")
                        code_content = code_container.text
                        
                        # Add data to batch
                        submission_data = {
                            "submission_id": submission_id,
                            "username": username,
                            "problem_name": problem_name,
                            "language": language,
                            "code": code_content,
                            "page": page_num,
                            "url": current_url
                        }
                        
                        page_submissions.append(submission_data)
                        log_message(f"Successfully extracted submission {submission_id}")
                        
                        # Close the tab after processing to free memory
                        driver.close()
                        
                    except Exception as e:
                        log_message(f"Error processing submission: {e}")
                        try:
                            # Take screenshot on error for debugging
                            driver.save_screenshot(os.path.join(output_dir, f"error_page{page_num}_batch{batch_start//batch_size + 1}.png"))
                        except:
                            pass
                        
                        # Close the tab even if there was an error
                        try:
                            driver.close()
                        except:
                            pass
                
                # Switch back to the main window after processing the batch
                driver.switch_to.window(main_window)
                
                # Short break between batches
                time.sleep(3)
            
            # Save the results for this page
            if page_submissions:
                total_submissions += len(page_submissions)
                page_csv_path = os.path.join(output_dir, f"hackerrank_submissions_page{page_num:03d}.csv")
                save_to_csv(page_submissions, page_csv_path)
                log_message(f"Saved {len(page_submissions)} submissions from page {page_num}")
            
            # Prepare for next page
            if page_num < max_pages:
                # Random wait between pages to avoid detection
                wait_time = random.uniform(wait_between_pages[0], wait_between_pages[1])
                log_message(f"Waiting {wait_time:.1f} seconds before navigating to the next page")
                time.sleep(wait_time)
                
                # Try to find and click the next page button - UPDATED SELECTOR
                try:
                    # Look for the link to the next page based on the current page number
                    current_page = page_num
                    next_page = current_page + 1
                    next_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, f"a.backbone[data-page='{next_page}']"))
                    )
                    next_button.click()
                    log_message(f"Navigating to page {page_num + 1}")
                    time.sleep(5)  # Wait for next page to load
                except (TimeoutException, NoSuchElementException) as e:
                    log_message(f"Error navigating to next page: {e}")
                    break
        
        except Exception as e:
            log_message(f"Error processing page {page_num}: {e}")
            # Try to recover and continue
            try:
                driver.refresh()
                time.sleep(5)
                log_message("Page refreshed. Trying to continue...")
            except:
                log_message("Could not recover. Stopping scraper.")
                break
    
    # Create a combined CSV of all pages at the end
    combine_csv_files(output_dir)
    
    log_message(f"Scraping complete! Total submissions scraped: {total_submissions}")
    return total_submissions

def save_to_csv(data, filename):
    """Save scraped data to a CSV file"""
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as file:
            # Define CSV columns
            fieldnames = ["submission_id", "username", "problem_name", "language", "code", "page", "url"]
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            
            # Write header and data
            writer.writeheader()
            writer.writerows(data)
        
        print(f"Successfully saved {len(data)} submissions to {filename}")
    except Exception as e:
        print(f"Error saving CSV file {filename}: {e}")

def combine_csv_files(directory, output_file="all_submissions.csv"):
    """Combine all individual page CSV files into one master file"""
    all_data = []
    csv_files = [f for f in os.listdir(directory) if f.startswith("hackerrank_submissions_page") and f.endswith(".csv")]
    
    for csv_file in sorted(csv_files):
        file_path = os.path.join(directory, csv_file)
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                all_data.extend(list(reader))
            print(f"Added data from {csv_file}")
        except Exception as e:
            print(f"Error reading {csv_file}: {e}")
    
    if all_data:
        output_path = os.path.join(directory, output_file)
        with open(output_path, 'w', newline='', encoding='utf-8') as file:
            fieldnames = list(all_data[0].keys())
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(all_data)
        print(f"Successfully combined {len(csv_files)} files into {output_file} with {len(all_data)} total submissions")
    else:
        print("No data found to combine")

# Main execution
if __name__ == "__main__":
    # Initialize the WebDriver (if running as script)
    # options = webdriver.ChromeOptions()
    # driver = webdriver.Chrome(options=options)
    # driver.get("https://www.hackerrank.com/contests/tcet-shastra-coding-contest-13-a/judge/submissions/2")
    
    # Run the scraper with existing driver
    scrape_hackerrank_with_pagination(
        driver=driver,
        max_pages=190,              # Adjust as needed (190 pages total)
        batch_size=3,               # Process 3 submissions at a time
        output_dir="hackerrank_data",
        wait_between_pages=(5,8)  # Random wait between pages
    )

[2025-04-06 12:56:16] 
==== Processing Page 1/190 ====
[2025-04-06 12:56:19] Found 10 'View' buttons on page 1
[2025-04-06 12:56:19] Processing batch 1 of 4
[2025-04-06 12:56:20] Opened submission 1/10 (ID: 1390888484)
[2025-04-06 12:56:21] Opened submission 2/10 (ID: 1390888164)
[2025-04-06 12:56:22] Opened submission 3/10 (ID: 1390888085)
[2025-04-06 12:56:27] Successfully extracted submission 1390888484
[2025-04-06 12:56:28] Successfully extracted submission 1390888085
[2025-04-06 12:56:28] Successfully extracted submission 1390888164
[2025-04-06 12:56:31] Processing batch 2 of 4
[2025-04-06 12:56:32] Opened submission 4/10 (ID: 1390887811)
[2025-04-06 12:56:33] Opened submission 5/10 (ID: 1390887752)
[2025-04-06 12:56:34] Opened submission 6/10 (ID: 1390887531)
[2025-04-06 12:56:39] Successfully extracted submission 1390887531
[2025-04-06 12:56:40] Successfully extracted submission 1390887752
[2025-04-06 12:56:40] Successfully extracted submission 1390887811
[2025-04-06 12:56:43] P

In [1]:
import csv
import os
import re
import pandas as pd
from tqdm import tqdm

def clean_code_section(code_text):
    """
    Clean code by removing standalone line numbers.
    The code format has line numbers as separate lines:
    1
    import java.util.Scanner;
    2
    3
    """
    if pd.isna(code_text) or code_text == '':
        return ''
        
    lines = code_text.split('\n')
    cleaned_lines = []
    
    # Regex to match lines that contain only numbers (possibly with whitespace)
    standalone_number_pattern = re.compile(r'^\s*\d+\s*$')
    
    for line in lines:
        # Skip lines that are just numbers
        if not standalone_number_pattern.match(line):
            cleaned_lines.append(line)
    
    # Print a small sample for debugging
    print("\nCleaning example:")
    print("BEFORE (first few lines):")
    for i in range(min(5, len(lines))):
        print(f"  {lines[i]}")
    
    print("\nAFTER (first few lines):")
    for i in range(min(5, len(cleaned_lines))):
        print(f"  {cleaned_lines[i]}")
    
    return '\n'.join(cleaned_lines)

def process_csv_file(input_file, output_file):
    """
    Process a CSV file containing HackerRank submissions.
    """
    try:
        print(f"Reading {input_file}...")
        df = pd.read_csv(input_file, encoding='utf-8')
        
        if 'code' not in df.columns:
            print(f"Error: 'code' column not found in {input_file}")
            return False
        
        print(f"Cleaning {len(df)} code submissions in {os.path.basename(input_file)}...")
        
        # Process a sample first to verify
        if not df.empty:
            sample_index = 0
            sample_orig = df.iloc[sample_index]['code']
            cleaned_sample = clean_code_section(sample_orig)
            
            # Save samples for verification
            output_dir = os.path.dirname(output_file)
            with open(os.path.join(output_dir, "sample_original.txt"), "w", encoding="utf-8") as f:
                f.write(sample_orig)
            with open(os.path.join(output_dir, "sample_cleaned.txt"), "w", encoding="utf-8") as f:
                f.write(cleaned_sample)
            
            print("\nSaved sample files for verification")
        
        # Clean all code entries
        df['code'] = df['code'].apply(clean_code_section)
        
        # Save the cleaned data
        df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"Successfully saved cleaned data to {output_file}")
        return True
        
    except Exception as e:
        print(f"Error processing {input_file}: {e}")
        return False

def main():
    # Directory paths
    INPUT_DIR = "hackerrank_data"
    OUTPUT_DIR = "hackerrank_data_cleaned"
    
    # Create output directory if needed
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"Created output directory: {OUTPUT_DIR}")
    
    # Process the all_submissions.csv file
    input_path = os.path.join(INPUT_DIR, "all_submissions.csv")
    output_path = os.path.join(OUTPUT_DIR, "cleaned_all_submissions.csv")
    
    if not os.path.exists(input_path):
        print(f"Error: {input_path} does not exist")
        return
    
    process_csv_file(input_path, output_path)
    
    print("\nVerification:")
    if os.path.exists(output_path):
        orig_df = pd.read_csv(input_path)
        cleaned_df = pd.read_csv(output_path)
        print(f"Original rows: {len(orig_df)}")
        print(f"Cleaned rows: {len(cleaned_df)}")
        print("Cleaning complete!")

if __name__ == "__main__":
    main()

Error: hackerrank_data\all_submissions.csv does not exist


In [3]:
import pandas as pd
import os
import hashlib
from collections import defaultdict
from tqdm import tqdm

def detect_plagiarism(csv_file, output_dir="plagiarism_results"):
    """
    Detect plagiarism by finding identical code across different submission IDs.
    
    Args:
        csv_file: Path to the cleaned CSV file with submissions
        output_dir: Directory to save plagiarism results
    """
    print(f"Reading submissions from {csv_file}...")
    df = pd.read_csv(csv_file)
    
    if 'code' not in df.columns or 'submission_id' not in df.columns:
        print("Error: CSV must contain 'code' and 'submission_id' columns")
        return
    
    print(f"Analyzing {len(df)} submissions for plagiarism...")
    
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Step 1: Group by problem to compare only within the same problem
    if 'problem_name' in df.columns:
        print("Grouping submissions by problem name...")
        problem_groups = df.groupby('problem_name')
    else:
        # If no problem_name column, treat all submissions as one group
        print("No problem_name column found, comparing all submissions together")
        problem_groups = [("all_problems", df)]
    
    # Step 2: Find identical code within each problem
    results = []
    all_plagiarism_groups = []
    
    # Process each problem group
    for problem_name, group_df in tqdm(problem_groups):
        # Hash the code to find duplicates efficiently
        code_hash_to_submissions = defaultdict(list)
        
        # Process each submission in the group
        for _, row in group_df.iterrows():
            submission_id = row['submission_id']
            code = row['code']
            username = row.get('username', 'unknown')
            language = row.get('language', 'unknown')
            
            # Skip empty code
            if pd.isna(code) or code.strip() == '':
                continue
                
            # Normalize code by removing whitespace to catch minor formatting differences
            normalized_code = '\n'.join([line.strip() for line in code.split('\n') if line.strip()])
            
            # Hash the normalized code
            code_hash = hashlib.md5(normalized_code.encode()).hexdigest()
            
            # Store submission details
            code_hash_to_submissions[code_hash].append({
                'submission_id': submission_id,
                'username': username,
                'language': language
            })
        
        # Find groups with multiple submissions (potential plagiarism)
        plagiarism_groups = {
            code_hash: submissions for code_hash, submissions in code_hash_to_submissions.items()
            if len(submissions) > 1
        }
        
        if plagiarism_groups:
            # Get the original code for each group
            for code_hash, submissions in plagiarism_groups.items():
                # Find the code for this hash
                for _, row in group_df.iterrows():
                    code = row['code']
                    if pd.isna(code) or code.strip() == '':
                        continue
                        
                    normalized_code = '\n'.join([line.strip() for line in code.split('\n') if line.strip()])
                    if hashlib.md5(normalized_code.encode()).hexdigest() == code_hash:
                        # Found a match, add code to the group and break
                        plagiarism_group = {
                            'problem_name': problem_name,
                            'submissions': submissions,
                            'identical_submissions_count': len(submissions),
                            'code': code
                        }
                        all_plagiarism_groups.append(plagiarism_group)
                        break
            
            # Record summary for this problem
            results.append({
                'problem_name': problem_name,
                'total_submissions': len(group_df),
                'plagiarism_groups': len(plagiarism_groups),
                'total_plagiarized_submissions': sum(len(subs) for subs in plagiarism_groups.values())
            })
    
    # Step 3: Generate reports
    # Summary report
    summary_df = pd.DataFrame(results)
    summary_file = os.path.join(output_dir, "plagiarism_summary.csv")
    summary_df.to_csv(summary_file, index=False)
    print(f"Summary report saved to {summary_file}")
    
    # Detailed report with all plagiarism groups
    detailed_file = os.path.join(output_dir, "plagiarism_detailed.txt")
    with open(detailed_file, 'w', encoding='utf-8') as f:
        f.write("DETAILED PLAGIARISM REPORT\n")
        f.write("=========================\n\n")
        
        for i, group in enumerate(sorted(all_plagiarism_groups, key=lambda x: x['identical_submissions_count'], reverse=True)):
            f.write(f"Group {i+1}: {group['problem_name']} - {group['identical_submissions_count']} identical submissions\n")
            
            # Write submission details
            f.write("Submissions:\n")
            for sub in group['submissions']:
                f.write(f"  - ID: {sub['submission_id']}, User: {sub['username']}, Language: {sub['language']}\n")
            
            # Write code sample
            f.write("\nCode:\n")
            f.write("```\n")
            f.write(group['code'][:500])  # First 500 chars
            if len(group['code']) > 500:
                f.write("\n... (truncated) ...\n")
            else:
                f.write("\n")
            f.write("```\n\n")
            f.write("-" * 80 + "\n\n")
    
    print(f"Detailed report saved to {detailed_file}")
    
    # Export each plagiarism group to a separate CSV for easy analysis
    export_df_rows = []
    for group_index, group in enumerate(all_plagiarism_groups):
        problem_name = group['problem_name']
        for sub in group['submissions']:
            export_df_rows.append({
                'group_id': group_index + 1,
                'problem_name': problem_name,
                'submission_id': sub['submission_id'],
                'username': sub['username'],
                'language': sub['language'],
                'identical_submissions_in_group': len(group['submissions'])
            })
    
    export_df = pd.DataFrame(export_df_rows)
    export_file = os.path.join(output_dir, "plagiarism_groups.csv")
    export_df.to_csv(export_file, index=False)
    print(f"Plagiarism groups exported to {export_file}")
    
    # Print summary statistics
    total_plagiarism_groups = len(all_plagiarism_groups)
    total_plagiarized_submissions = sum(len(group['submissions']) for group in all_plagiarism_groups)
    
    print("\nPlagiarism Detection Results:")
    print(f"- Total submissions analyzed: {len(df)}")
    print(f"- Plagiarism groups found: {total_plagiarism_groups}")
    print(f"- Submissions involved in plagiarism: {total_plagiarized_submissions}")
    
    if total_plagiarism_groups > 0:
        # Find the largest plagiarism group
        largest_group = max(all_plagiarism_groups, key=lambda x: x['identical_submissions_count'])
        print(f"- Largest plagiarism group: {largest_group['identical_submissions_count']} identical submissions " +
              f"for problem '{largest_group['problem_name']}'")
    
    return export_df

if __name__ == "__main__":
    # Directory paths
    INPUT_DIR = "hackerrank_data_cleaned"
    OUTPUT_DIR = "plagiarism_results"
    
    # Input file (cleaned submissions)
    input_file = os.path.join(INPUT_DIR, "cleaned_all_submissions.csv")
    
    if not os.path.exists(input_file):
        print(f"Error: {input_file} does not exist")
    else:
        detect_plagiarism(input_file, OUTPUT_DIR)

Error: hackerrank_data_cleaned\cleaned_all_submissions.csv does not exist
